In [37]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
from datetime import date

In [65]:
from bokeh.io import output_notebook, show, output_file
output_notebook()

Loading BokehJS ...

In [67]:
from bokeh.plotting import figure, save
from bokeh.models import BoxAnnotation, Span

In [76]:
df = pd.read_csv('../data/parsed/cases.csv', parse_dates=['date'])

In [77]:
df = df[df['region']=='Anchorage'].copy()

In [78]:
df['smoothed'] = df['cases'].rolling(7,
        win_type='hamming',
        min_periods=3,
        center=True).mean().round()

In [79]:
p1 = figure(x_axis_type='datetime', title='Anchorage Daily Cases', aspect_ratio=1.75)
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Daily Total'
p1.circle(df['date'], df['cases'], color='gray', alpha=0.7)
p1.line(df['date'], df['smoothed'], color='blue', alpha=0.8)

# Indicate the stay-at-home duration
p1.add_layout(BoxAnnotation(left=parse('16 MAR 2020'), 
                            right=parse('27 APR 2020'), 
                            fill_color='red', 
                            fill_alpha=0.025))

# Indicate the mask mandate duration
p1.add_layout(BoxAnnotation(left=parse('29 JUN 2020'), 
                            right=date.today(), 
                            fill_color='orange', 
                            fill_alpha=0.05))

# add Memorial Day Weekend start
p1.add_layout(Span(location=parse('22 MAY 2020'),
                          dimension='height', line_color='black',
                          line_dash='dashed', line_width=1))

# add Summer Solistce (only in Alaska...)
p1.add_layout(Span(location=parse('20 JUN 2020'),
                          dimension='height', line_color='black',
                          line_dash='dashed', line_width=1))

# add Independence Day Weekend
p1.add_layout(Span(location=parse('3 JUL 2020'),
                          dimension='height', line_color='black',
                          line_dash='dashed', line_width=1))


show(p1)
output_file('../plots/Anchorage Cases.html', mode='inline')
save(p1)

'C:\\Users\\crock\\Documents\\GitHub\\akcoviddata\\plots\\Anchorage Cases.html'

,date,cases,recoveries,deaths,hospitalizations,region,active_cases
0,2020-03-03,3,0,0,0,Statewide,3
1,2020-03-04,1,0,0,0,Statewide,4
2,2020-03-05,1,0,0,0,Statewide,5
3,2020-03-06,1,0,0,1,Statewide,6
4,2020-03-07,1,0,0,0,Statewide,7
